# Librerias

In [1]:
random_state = 43992294
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MultiLabelBinarizer

from xgboost import XGBClassifier
from sklearn.preprocessing import FunctionTransformer

import ast

import warnings

from tqdm.auto import tqdm

from sklearn.feature_selection import RFECV

from sklearn.model_selection import StratifiedKFold, cross_val_score

from itertools import chain

import dask.dataframe as dd

from sklearn.utils import resample

# Cython imports
from tools import agrupar_categorias_cython, custom_one_hot_encoder_cython, boolean_features_ohe_cython, agrupar_edades_cython, expand_action_list_0_cython

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Funciones

In [2]:
warnings.filterwarnings('ignore')
tqdm.pandas()

# Agrega filas que contienen categorías desconocidas o raras (poca frecuencia) al conjunto de entrenamiento
def augment_train_data(main_train_df, supplementary_df, umbral_raras=100):
    """
    Agrega filas del dataset suplementario al conjunto de entrenamiento principal
    basándose en categorías desconocidas y raras, evitando la duplicación de filas.

    Parámetros:
    - main_train_df (pd.DataFrame): DataFrame principal de entrenamiento.
    - supplementary_df (pd.DataFrame): DataFrame suplementario del cual se extraerán las filas.
    - umbral_raras (int): Umbral de frecuencia para considerar una categoría como rara.

    Retorna:
    - main_train_df (pd.DataFrame): DataFrame de entrenamiento actualizado.
    - categorias_desconocidas (dict): Diccionario actualizado de categorías desconocidas.
    - categorias_raras (dict): Diccionario actualizado de categorías raras.
    """
    # Definir columnas que no deseas tratar como categóricas
    columns_to_exclude = ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']

    # Identificar columnas categóricas excluyendo las especificadas
    categorical_features = main_train_df.select_dtypes(include=['object']).columns.tolist()
    categorical_features = [col for col in categorical_features if col not in columns_to_exclude]
    
    # Crear un diccionario para almacenar las categorías desconocidas por columna
    categorias_desconocidas = {}
    
    # Iterar a través de cada columna categórica para identificar categorías desconocidas
    for columna in categorical_features:
        # Obtener las categorías únicas en el conjunto de entrenamiento
        categorias_train = set(main_train_df[columna].dropna().unique())
        
        # Obtener las categorías únicas en el dataset suplementario
        categorias_suplementario = set(supplementary_df[columna].dropna().unique())
        
        # Identificar las categorías en el dataset suplementario que no están en el entrenamiento
        desconocidas = categorias_suplementario - categorias_train
        
        # Almacenar las categorías desconocidas en el diccionario como una lista
        categorias_desconocidas[columna] = list(desconocidas)
    
    # Inicializar el diccionario para almacenar las categorías raras por columna
    categorias_raras = {}
    
    # Identificar categorías raras en el conjunto de entrenamiento
    for columna in categorical_features:
        # Contar la frecuencia de cada categoría
        frecuencia = main_train_df[columna].value_counts()
        
        # Identificar categorías que aparecen menos de umbral_raras veces
        raras = frecuencia[frecuencia < umbral_raras].index.tolist()
        
        # Almacenar en el diccionario
        categorias_raras[columna] = raras
    
    # Crear una máscara booleana para filas con categorías desconocidas o raras
    mask_desconocidas = pd.Series([False] * len(supplementary_df))
    mask_raras = pd.Series([False] * len(supplementary_df))
    
    for columna in categorical_features:
        # Actualizar la máscara para categorías desconocidas
        if categorias_desconocidas[columna]:
            mask_desconocidas = mask_desconocidas | supplementary_df[columna].isin(categorias_desconocidas[columna])
        
        # Actualizar la máscara para categorías raras
        if categorias_raras[columna]:
            mask_raras = mask_raras | supplementary_df[columna].isin(categorias_raras[columna])
    
    # Combinar ambas máscaras
    mask_total = mask_desconocidas | mask_raras
    
    # Filtrar filas únicas a agregar
    filas_a_agregar = supplementary_df[mask_total].drop_duplicates()
    
    # Mostrar información de agregación
    total_agregadas = len(filas_a_agregar)
    print(f"\nAgregando {total_agregadas} filas del dataset suplementario basadas en categorías desconocidas o raras.")
    
    # Agregar las filas al conjunto de entrenamiento
    main_train_df = pd.concat([main_train_df, filas_a_agregar], ignore_index=True)
    
    # Actualizar los diccionarios eliminando las categorías que ya han sido agregadas
    for columna in categorical_features:
        # Actualizar categorías desconocidas
        if categorias_desconocidas[columna]:
            categorias_agregadas = filas_a_agregar[columna].unique().tolist()
            categorias_desconocidas[columna] = [cat for cat in categorias_desconocidas[columna] if cat not in categorias_agregadas]
        
        # Actualizar categorías raras
        if categorias_raras[columna]:
            # Recontar la frecuencia después de agregar
            frecuencia = main_train_df[columna].value_counts()
            nuevas_raras = frecuencia[frecuencia < umbral_raras].index.tolist()
            categorias_raras[columna] = nuevas_raras
    
    return main_train_df, categorias_desconocidas, categorias_raras

# Agrega como columnas binarias las listas de la columna 'auction_list_0', 'action_list_1' y 'action_list_2'
def expand_list_dummies_cython(df, column, delimiter='|', prefix=None, suffix=None):
    """
    Expande una columna que contiene listas en múltiples columnas binarias usando un one-hot encoder optimizado con Cython.
    
    Parameters:
    - df (pd.DataFrame): DataFrame de pandas.
    - column (str): Nombre de la columna a expandir.
    - delimiter (str): Delimitador a usar en get_dummies (por defecto '|').
    - prefix (str, optional): Prefijo para las nuevas columnas binarias.
    - suffix (str, optional): Sufijo para las nuevas columnas binarias.
    
    Returns:
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas binarias añadidas y la columna original eliminada.
    """
    print(f"Comenzando la expansión de la columna: '{column}'")
    
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    
    # Reemplazar NaN por listas vacías
    print(f"Reemplazando NaN en la columna '{column}' por listas vacías.")
    df[column] = df[column].fillna('[]')
    
    # Definir la función de parsing con impresión de errores
    def parse_list(x):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                # Convertir todos los elementos a strings
                return [str(item) for item in parsed]
            else:
                # Si no es una lista, tratar como un solo elemento
                return [str(x)]
        except (ValueError, SyntaxError):
            # En caso de error al parsear, retornar una lista vacía
            return []
    
    # Aplicar la función de parsing con una barra de progreso
    df[column] = df[column].progress_apply(parse_list)
    
    # Convertir la columna en una lista de listas para pasarla a la función Cythonizada
    data_list = df[column].tolist()
    
    # Llamar a la función optimizada en Cython
    unique_categories, binary_matrix = custom_one_hot_encoder_cython(data_list)
    print(f"Codificación completada. {len(unique_categories)} categorías únicas encontradas.")
    
    # Crear un DataFrame binario usando la matriz devuelta por Cython
    binary_df = pd.DataFrame(binary_matrix, index=df.index, columns=unique_categories)

    if prefix:
        binary_df = binary_df.add_prefix(f"{prefix}_")
    if suffix:
        binary_df = binary_df.add_suffix(f"_{suffix}")
    
    # Concatenar las columnas binarias al DataFrame original con una barra de progreso
    print("Concatenando las columnas binarias al DataFrame original.")
    for col in tqdm(binary_df.columns, desc="Concatenando columnas binarias"):
        if col in df.columns:
            df[col] = np.where((df[col] == 1) | (binary_df[col] == 1), 1, 0)
        else:
            df[col] = binary_df[col]
    
    # Eliminar la columna original ya que ha sido expandida
    print(f"Eliminando la columna original '{column}' del DataFrame.")
    df = df.drop(columns=[column])
    
    print(f"Expansión de la columna '{column}' completada exitosamente.\n")
    
    return df

# One-hot encode de columnas booleanas utilizando Cython
def boolean_features_ohe(df, columns_to_encode=['auction_boolean_0', 'auction_boolean_1', 'auction_boolean_2']):
    """
    Realiza one-hot encoding en columnas booleanas especificadas utilizando una función optimizada con Cython.
    Además, muestra el progreso del procesamiento utilizando tqdm y añade comentarios explicativos.

    Parámetros:
    - df (pd.DataFrame): DataFrame original que contiene las columnas booleanas a codificar.
    - columns_to_encode (list): Lista de nombres de columnas booleanas a codificar.

    Retorna:
    - df_expanded (pd.DataFrame): DataFrame con las nuevas columnas codificadas añadidas y las columnas booleanas originales eliminadas.
    """
    # Copiar el DataFrame para evitar modificar el original
    df = df.copy()
    print("Inicio del proceso de one-hot encoding para las columnas booleanas especificadas.")

    # Paso 1: Encontrar todos los valores únicos en las columnas a codificar
    unique_values_set = set()
    print("Recopilando valores únicos de las columnas a codificar:")
    for col in tqdm(columns_to_encode, desc="Procesando columnas para valores únicos"):
        unique_vals_col = df[col].dropna().unique()
        unique_values_set.update(unique_vals_col)
    unique_values = sorted(unique_values_set)
    print(f"Valores únicos encontrados: {unique_values}")

    # Paso 2: Convertir las columnas a listas de listas para ser procesadas en Cython
    list_data = []
    print("Convirtiendo las columnas booleanas a listas de listas para Cython:")
    for col in tqdm(columns_to_encode, desc="Convertir columnas a listas"):
        column_list = df[col].astype(str).tolist()  # Mantener los valores como strings
        list_data.append(column_list)
    print("Conversión completada.")

    # Paso 3: Procesar los datos con la función optimizada en Cython
    print("Realizando one-hot encoding utilizando la función optimizada en Cython:")
    ohe_result = boolean_features_ohe_cython(list_data, unique_values)
    print("One-hot encoding completado.")

    # Paso 4: Convertir el resultado de Cython a un DataFrame, alineando el índice con df
    print("Creando el DataFrame de columnas codificadas:")
    ohe_df = pd.DataFrame(ohe_result, columns=unique_values, index=df.index)
    print(f"DataFrame de one-hot encoding creado con {len(ohe_df.columns)} columnas y {ohe_df.shape[0]} filas.")

    # Paso 5: Concatenar las nuevas columnas codificadas al DataFrame original
    print("Concatenando las columnas codificadas al DataFrame original:")
    df_expanded = pd.concat([df, ohe_df], axis=1)
    print(f"Concatenación completada. El DataFrame ahora tiene {df_expanded.shape[1]} columnas y {df_expanded.shape[0]} filas.")

    # Paso 6: Eliminar las columnas booleanas originales del DataFrame
    print("Eliminando las columnas booleanas originales del DataFrame:")
    df_expanded.drop(columns=columns_to_encode, inplace=True)
    print(f"Columnas eliminadas: {columns_to_encode}")

    print("Proceso de one-hot encoding finalizado exitosamente.\n")

    return df_expanded

# Extensión de características temporales (día de la semana, momento del día, etc.) y festividades
def time_features_extension(df):
    """
    Procesa las características temporales del DataFrame y agrega nuevas columnas derivadas relacionadas con el tiempo y festividades.
    """

    # Convertir 'auction_time' de timestamp a una fecha legible
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s')

    # Reemplazar NaN en 'timezone_offset' por 0
    df['timezone_offset'] = df['timezone_offset'].fillna(0)

    # Ajustar la hora según el 'timezone_offset' para obtener la hora local
    df['auction_time_local'] = df.apply(
        lambda row: row['auction_time'] + pd.DateOffset(hours=row['timezone_offset']), axis=1
    )

    # Crear la columna 'week_day' (1 para lunes, 7 para domingo)
    df['week_day'] = df['auction_time_local'].dt.weekday + 1

    # Crear la columna 'moment_of_the_day' (1 para temprano, 2 para tarde, 3 para noche)
    df['moment_of_the_day'] = pd.cut(df['auction_time_local'].dt.hour, 
                                     bins=[0, 12, 18, 24], labels=[1, 2, 3], include_lowest=True, right=False)

    # Eliminar las columnas originales 'auction_time', 'timezone_offset' y 'auction_time_local'
    df.drop(columns=['auction_time', 'timezone_offset', 'auction_time_local'], inplace=True)

    return df

# Agrupación de edades en rangos numéricos
def age_group(df, columna_edad):
    """
    Agrupa las edades en rangos numéricos utilizando Cython para mejorar el rendimiento.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna de edades.
    - columna_edad (str): Nombre de la columna que contiene las edades.

    Retorna:
    - df (pd.DataFrame): DataFrame con la nueva columna 'age_group' que representa el rango de edad.
    """
    # Convertir la columna de edad a una lista
    edades = df[columna_edad].tolist()

    # Usar la función Cythonizada para agrupar las edades
    df['age_group'] = agrupar_edades_cython(edades)

    # Eliminar la columna original de edades
    df.drop(columns=[columna_edad], inplace=True)

    return df

# Agrupo action_list_0 a auction_list_0
def expand_action_list_0(df):
    """
    Expande la columna 'action_list_0' en valores únicos y marca con 1 las columnas existentes o las crea si es necesario.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna 'action_list_0' y otras columnas de listas ya expandidas.

    Retorna:
    - df (pd.DataFrame): DataFrame actualizado con las columnas de valores únicos de 'action_list_0'.
    """

    # Convertir la columna 'action_list_0' y las columnas existentes a listas
    action_list_0 = df['action_list_0'].tolist()
    existing_columns = df.columns.tolist()
    
    # Inicializar la matriz actual
    current_matrix = df.values.tolist()

    # Llamar a la función Cythonizada
    updated_matrix = expand_action_list_0_cython(action_list_0, existing_columns, current_matrix)

    # Convertir la matriz actualizada de vuelta a un DataFrame
    df_updated = pd.DataFrame(updated_matrix, columns=existing_columns)

    # Eliminar la columna 'action_list_0'
    df_updated.drop(columns=['action_list_0'], inplace=True)

    return df_updated

# Concateno las categorias de cada nivel
def create_level_combination(df):
    """
    Creates a new column 'level_combination' by concatenating the first three characters 
    of each 'action_categorical' level columns and removes the original level columns.

    Parameters:
    - df (pd.DataFrame): The input DataFrame containing the columns:
      'action_categorical_0', 'action_categorical_1', 'action_categorical_2', 
      'action_categorical_3', 'action_categorical_4'.

    Returns:
    - pd.DataFrame: The DataFrame with the new 'level_combination' column and without the original level columns.
    """
    level_columns = [
        'action_categorical_0',
        'action_categorical_1',
        'action_categorical_2',
        'action_categorical_3',
        'action_categorical_4'
    ]
    df['level_combination'] = df[level_columns].astype(str).apply(
        lambda x: ''.join([s[:3] for s in x]), axis=1
    )
    df.drop(columns=level_columns, inplace=True)
    return df

# Heigh x Width a columna
def hxw_column(df):
    """
    Crea una nueva columna 'hxw' multiplicando 'creative_height' y 'creative_width'.
    Si alguno de los dos tiene un NaN, 'hxw' se establece en 0.
    Elimina las columnas originales 'creative_height' y 'creative_width'.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene las columnas 'creative_height' y 'creative_width'.
    
    Retorna:
    - pd.DataFrame: DataFrame con la nueva columna 'hxw' añadida y las columnas originales eliminadas.
    """
    df['hxw'] = df['creative_height'] * df['creative_width']
    df.loc[df['creative_height'].isna() | df['creative_width'].isna(), 'hxw'] = 0
    df.drop(columns=['creative_height', 'creative_width'], inplace=True)
    return df

# Gender to number
def encode_gender(df):
    """
    Reemplaza los valores de la columna 'gender' de la siguiente manera:
    'f' -> 1, 'm' -> 2, 'o' -> 0 y NaN -> -1.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene la columna 'gender'.
    
    Retorna:
    - pd.DataFrame: DataFrame con la columna 'gender' codificada.
    """
    df['gender'] = df['gender'].map({'f': 1, 'm': 2, 'o': 0}).fillna(-1).astype(int)
    return df

# creative_categorical_11, creative_categorical_9 y creative_categorical_10 a dos columnas
def creatives2unique(df):
    """
    Crea o actualiza dos columnas en el DataFrame, una para cada valor único en las columnas
    'creative_categorical_11', 'creative_categorical_10', y 'creative_categorical_9'.
    Si las columnas ya existen, actualiza los valores a 1 donde ese valor aparece en alguna
    de las tres columnas en esa fila.

    Parámetros:
    - df (pd.DataFrame): DataFrame que contiene las columnas 'creative_categorical_11',
                         'creative_categorical_10', y 'creative_categorical_9'.

    Retorna:
    - pd.DataFrame: DataFrame con las nuevas columnas añadidas o actualizadas.
    """
    unique_values = {'65dcab89', '43c867fd'}
    columns_to_check = [
        'creative_categorical_11',
        'creative_categorical_10',
        'creative_categorical_9'
    ]

    for val in unique_values:
        if val in df.columns:
            # Si la columna ya existe, actualizamos los valores a 1 donde corresponde
            df[val] = df[val] | df[columns_to_check].eq(val).any(axis=1).astype(int)
        else:
            # Si no existe, creamos la columna con 1 donde corresponde
            df[val] = df[columns_to_check].eq(val).any(axis=1).astype(int)

    # Eliminar las columnas originales
    df.drop(columns=columns_to_check, inplace=True)
    
    return df

# Juntar todas las variables categoricas y hacer OHE
def process_combineta(df):
    """
    Procesa las columnas proporcionadas en combineta, creando un set con valores únicos,
    y generando columnas binarias para cada uno de esos valores. Si la columna ya existe,
    actualiza las filas con un 1 donde corresponda. Luego, elimina las columnas originales.

    Parámetros:
    - df (pd.DataFrame): DataFrame original.
    - combineta_columns (list): Lista de columnas a procesar.

    Retorna:
    - df (pd.DataFrame): DataFrame con las columnas binarias añadidas y las columnas originales eliminadas.
    """
    combineta_columns = ['creative_categorical_0', 'creative_categorical_5', 'auction_categorical_0', 'auction_categorical_1', 'auction_categorical_11', 'auction_categorical_7', 'auction_categorical_8', 'auction_categorical_9', 'action_categorical_6', 'action_categorical_7', 'auction_categorical_3', 'auction_categorical_4', 'auction_categorical_5', 'auction_categorical_6', 'auction_categorical_10', 'auction_categorical_12', 'creative_categorical_1', 'creative_categorical_12', 'creative_categorical_2', 'creative_categorical_3', 'creative_categorical_4', 'creative_categorical_6', 'creative_categorical_7', 'creative_categorical_8']

    # Unir todas las columnas de combineta en una sola columna de listas
    df['combined_combineta'] = df[combineta_columns].astype(str).agg(
        lambda x: '[' + ', '.join([f"'{str(item).strip()}'" for item in x if item != 'nan']) + ']', axis=1)

    # Usar la función expand_list_dummies_cython para descomponer la lista y crear las columnas binarias
    df = expand_list_dummies_cython(df, 'combined_combineta')
    
    df.drop(columns=combineta_columns, inplace=True, errors='raise')
    
    return df

# Procesamiento optimizado de un DataFrame
def process_optimized(df):
    """
    Aplica una serie de transformaciones al DataFrame utilizando una función Cython optimizada.
    
    Parámetros:
    - df (pd.DataFrame): DataFrame a procesar.
    
    Retorna:
    - df (pd.DataFrame): DataFrame procesado.
    """
    # Definir el número total de pasos para la barra de progreso
    total_steps = 14
    
    # Inicializar la barra de progreso
    with tqdm(total=total_steps, desc="Procesando DataFrame", unit="paso") as pbar:
        
        print("Comenzando el procesamiento optimizado del DataFrame.")
        print("Eliminando columnas innecesarias.")
        df = df.drop('action_categorical_5', axis=1)
        df = df.drop('auction_categorical_2', axis=1)
        pbar.update(1)

        print("Expansión de columnas temporales")
        df = time_features_extension(df)
        pbar.update(1)

        df = age_group(df, 'auction_age')
        pbar.update(1)

        print("Agrupando columnas de nivel")
        df = create_level_combination(df)
        pbar.update(1)

        print("Modificando columnas de genero")
        df = encode_gender(df)
        pbar.update(1)
        
        print("Modificando columna de video")
        df['has_video'] = df['has_video'].apply(lambda x: 1 if x == True else 0)
        pbar.update(1)

        print("Juntando medidas")
        df = hxw_column(df)
        pbar.update(1)

        print("Expansión de columnas booleanas.")
        df = boolean_features_ohe(df)
        pbar.update(1)

        print("Creando columnas de creatividad")
        df = creatives2unique(df)
        pbar.update(1)
        
        columns_to_expand = ['auction_list_0', 'action_list_1']

        # Bucle para realizar las operaciones
        for col in columns_to_expand:
            print(f"Expansión de columnas de listas para {col}.")
            
            # Crear la variable 'idx_position' con la última columna antes de la expansión
            idx_position = df.columns.get_loc(df.columns[-1])
            
            # Expansión de la columna
            df = expand_list_dummies_cython(df, col)
            pbar.update(1)

            if col == 'action_list_1':
                df = expand_list_dummies_cython(df,'action_list_2')
                pbar.update(1)
            
            if col == 'auction_list_0':
                # Bucle para recorrer las columnas que empiezan con 'AND' o 'APL'
                if 'AND-APL' not in df.columns:
                    df['AND-APL'] = 0  # Inicializar la columna 'AND-APL'

                for column in df.columns:
                    if column.startswith('AND') or column.startswith('APL'):
                        # Poner un 1 en 'AND-APL' si la columna actual tiene un 1 en esa fila
                        df['AND-APL'] = df['AND-APL'] | df[column]

                # Eliminar todas las columnas que empiezan con 'AND' o 'APL' excepto la columna 'AND-APL'
                columns_to_drop = [column for column in df.columns if (column.startswith('AND') or column.startswith('APL')) and column != 'AND-APL']
                df.drop(columns=columns_to_drop, inplace=True)

                print("Complementamos con la columna 'action_list_0'")
                df = expand_action_list_0(df)
                pbar.update(1)

            # Crear la lista de columnas numéricas a partir de la siguiente columna después de 'idx'
            categorical_num = df.iloc[:, idx_position:].select_dtypes(include=['number']).columns

            # Eliminar columnas numéricas con menos de 1000 valores iguales a 1
            for column in categorical_num:
                if (df[column] == 1).sum() < 1000 and column != 'AND-APL':
                    df.drop(column, axis=1, inplace=True)
    
        
        print("Agrupando categorias poco frecuentes")

        categorical_str = df.select_dtypes(include=['object']).columns
        categorical_str = categorical_str[categorical_str != 'device_id']
        
        categorical_num = df.select_dtypes(include=['number']).columns
        
        # Convertir a matriz bidimensional
        data_matrix = df[categorical_str].values.tolist()
        data_matrix_cython = [list(row) for row in data_matrix]
        
        # Llamar a la función Cythonizada
        df_cython_data = agrupar_categorias_cython(
            categorical_features=categorical_str.tolist(),
            data=data_matrix_cython,
            umbral=1000  # Umbral de frecuencia para considerar una categoría como rara
        )

        # Reasignar los datos al DataFrame
        for i, col in enumerate(categorical_str):
            df[col] = [row[i] for row in df_cython_data]

        idx_position = df.columns.get_loc(df.columns[-1])
        
        df = process_combineta(df)

        # Crear la lista de columnas numéricas a partir de la siguiente columna después de 'idx'
        categorical_num = df.iloc[:, idx_position:].select_dtypes(include=['number']).columns

        # Eliminar columnas numéricas con menos de 1000 valores iguales a 1
        for column in categorical_num:
            if (df[column] == 1).sum() < 1000:
                df.drop(column, axis=1, inplace=True)

        pbar.update(1)

        
    return df

# Función para ajustar el tipo de datos de una columna para que Dask tome Nan como valor válido
def adjust_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'Int64'
    elif pd.api.types.is_float_dtype(dtype):
        return 'float64'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'boolean'
    else:
        return 'object'
    
# Procesamiento de datos con Dask
def process_data_with_dask(df, npartitions=10, meta_df=None):
    """
    Procesa un DataFrame utilizando Dask para distribuir el trabajo en varias particiones.
    Aplica la función process_optimized a cada partición del DataFrame.

    Parámetros:
    - df (pd.DataFrame): El DataFrame de pandas a procesar.
    - npartitions (int): Número de particiones en las que se dividirá el DataFrame para su procesamiento.

    Retorna:
    - final_df (pd.DataFrame): El DataFrame procesado y concatenado.
    """
    # Convertir el DataFrame de pandas a Dask con el número de particiones especificado
    dask_df = dd.from_pandas(df, npartitions=npartitions)
    
    # Crear el meta DataFrame con tipos ajustados
    meta = df.head(0).copy()
    for col in meta.columns:
        meta[col] = meta[col].astype(adjust_dtype(df[col].dtype))
        
    # Aplicar la función con map_partitions y especificar el meta
    dask_df = dask_df.map_partitions(
        lambda df_partition: process_optimized(df_partition),
        meta=meta
    )

    # Ejecutar el cálculo distribuido y convertir el resultado a pandas
    final_df = dask_df.compute()

    return final_df

# Función para calcular la estadística de Cramér's V
def cramers_v(confusion_matrix):
    """
    Calcula la estadística de Cramér's V para medir la asociación entre dos variables categóricas.

    Parameters:
    - confusion_matrix: Matriz de confusión (tabla de contingencia) entre dos variables.

    Returns:
    - Cramér's V: Valor entre 0 y 1 que indica la fuerza de la asociación.
    """
    # Calcular el estadístico chi-cuadrado
    chi2 = chi2_contingency(confusion_matrix, correction=False)[0]
    # Número total de observaciones
    n = confusion_matrix.sum().sum()
    # Obtener el número de filas y columnas de la matriz de confusión
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evitar dividir por cero
    # Calcular Cramér's V
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def handle_none(value):
    return 'None' if value is None else str(value)

# Lectura de datos

In [3]:
train_data = pd.read_csv('train_data_combined.csv')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [9]:
train_data_combined, _ = train_test_split(
    train_data, 
    train_size=50000, 
    random_state=random_state, 
    stratify=train_data['Label']
)

In [ ]:
train_data = process_optimized(train_data_combined)

# Graficos

## Correlaciones

In [ ]:
# Seleccionar las columnas numéricas
numeric_columns = train_data.select_dtypes(include=['number'])

# Verificar que 'Label' esté en las columnas numéricas
if 'Label' not in numeric_columns.columns:
    train_data['Label'] = train_data['Label'].astype(float)
    numeric_columns = train_data.select_dtypes(include=['number'])

# Análisis de correlación con 'Label'
correlation_with_label = numeric_columns.corr()['Label'].sort_values(key=abs, ascending=False)

# Imprimir correlaciones con 'Label'
print("Correlation of features with 'Label' (sorted by absolute value):")
for feature, corr in correlation_with_label.items():
    if feature != 'Label':
        print(f"{feature}: {corr:.4f}")

# Análisis de datos faltantes
missing_data = train_data.isnull().sum() / len(train_data) * 100
missing_data = missing_data.sort_values(ascending=False)

print("\nPercentage of Missing Data by Feature:")
for feature, percentage in missing_data.items():
    if percentage > 0:  # Solo imprimir características con datos faltantes
        print(f"{feature}: {percentage:.2f}%")

# Visualizar las mayores correlaciones con 'Label'
plt.figure(figsize=(12, 8))
top_correlations = correlation_with_label.drop('Label').abs().nlargest(15)
sns.barplot(x=top_correlations.values, y=top_correlations.index)
plt.title('Top 15 Features Correlated with Label')
plt.xlabel('Absolute Correlation')
plt.tight_layout()
plt.show()


# Comparación de modelos

## Recursive Feature Elimination

In [ ]:
# Separar las características (X) y el label (y)
X = train_data.drop('Label', axis=1)
y = train_data['Label']

In [ ]:
# Definir las columnas categóricas y numéricas
categorical_features_to_encode = X.select_dtypes(include=['object']).columns
numeric_features = X.select_dtypes(include=['number']).columns.tolist()

# Preprocesador común para imputación y codificación
common_preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos con la mediana
        ('num', SimpleImputer(strategy='median'), numeric_features),
        
        # Imputar valores categóricos con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('onehot', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'))
        ]), categorical_features_to_encode)
    ],
    remainder='drop'  # Excluir columnas no especificadas
)

# Definir el modelo XGBoost
model_xgb = XGBClassifier(
    random_state=random_state, 
    use_label_encoder=False, 
    eval_metric='auc'
)

# Crear el pipeline con RFECV y XGBoost
# Utilizamos StratifiedKFold para asegurar que se mantengan las proporciones de la clase
cv = StratifiedKFold(n_splits=5)

# Pipeline para XGBoost con RFECV
rfe_xgb = Pipeline([
    ('preprocessor', common_preprocessor),
    ('selector', RFECV(
        estimator=model_xgb,
        step=1,
        cv=cv,
        scoring='roc_auc',
        verbose=0,  # Desactiva la salida estándar de RFECV
        n_jobs=-1
    ))
])

# Ajustar el modelo con RFECV
print("Fitting XGBoost RFECV...")
rfe_xgb.named_steps['selector'].fit(X, y)

# Obtener el número óptimo de características
print('Optimal number of features (XGBoost model): {}'.format(rfe_xgb.named_steps['selector'].n_features_))

# Graficar la selección de características
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(rfe_xgb.named_steps['selector'].cv_results_['mean_test_score']) + 1),
         rfe_xgb.named_steps['selector'].cv_results_['mean_test_score'])
plt.title('Feature Number Selection (XGBoost)')
plt.xlabel("Number of Features Selected")
plt.ylabel("CV Score (ROC AUC)")
plt.show()

# Obtener los nombres de las características después del preprocesamiento
feature_names = (numeric_features +
                 rfe_xgb.named_steps['preprocessor']
                 .named_transformers_['cat']
                 .named_steps['onehot']
                 .get_feature_names_out(categorical_features_to_encode).tolist())

# Imprimir las características seleccionadas
print("\nFeatures selected by XGBoost:")
selected_features_xgb = [feature_names[i] for i, selected in enumerate(rfe_xgb.named_steps['selector'].support_) if selected]
print(selected_features_xgb)


## Backward Selection

In [7]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data.drop(columns='Label'),  # Características
    train_data['Label'],               # Variable objetivo
    test_size=0.2,                     # 20% para validación
    stratify=train_data['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state          # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [ ]:
# Identificar las columnas categóricas y numéricas
categorical_columns = X_train.select_dtypes(include=['object']).columns
numeric_columns = X_train.select_dtypes(include=['number']).columns

# Crear un preprocesador con imputación y One-Hot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        # Imputar valores numéricos faltantes con la media
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Imputar NaNs con la media
        ]), numeric_columns),
        
        # Imputar valores categóricos faltantes con 'Desconocido' y aplicar One-Hot Encoding
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),  # Imputar NaNs con 'Desconocido'
            ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # One-Hot Encoding
        ]), categorical_columns)
    ]
)

# Entrenamiento del modelo XGBoost
model_xgb = XGBClassifier(random_state=random_state)
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model_xgb)
])
pipeline_xgb.fit(X_train, y_train)

# Obtener los nombres de las características después del One-Hot Encoding
encoded_feature_names = pipeline_xgb.named_steps['preprocessor'].transformers_[1][1]['onehot'].get_feature_names_out(categorical_columns)
all_feature_names = np.concatenate([numeric_columns, encoded_feature_names])

# Crear un DataFrame para las importancias de XGBoost
feature_importances_df = pd.DataFrame({
    'Feature': all_feature_names,
    'XGBoostImportance': pipeline_xgb.named_steps['classifier'].feature_importances_
})

# Normalizar las importancias
feature_importances_df['XGBoostImportanceNorm'] = feature_importances_df['XGBoostImportance'] / feature_importances_df['XGBoostImportance'].sum()

# Ordenar por la importancia normalizada
feature_importances_df = feature_importances_df.sort_values(by='XGBoostImportanceNorm', ascending=False)

# Mostrar las 20 características más importantes
print(feature_importances_df.head(20))

# Visualización de la comparación con gráfico de barras (top 20 características)
plt.figure(figsize=(15, 10))
top_20 = feature_importances_df.head(20)
indices = np.arange(len(top_20))

plt.bar(indices, top_20['XGBoostImportanceNorm'], alpha=0.7)

plt.xlabel('Características')
plt.ylabel('Importancia Normalizada')
plt.title('Top 20 Características según XGBoost')
plt.xticks(indices, top_20['Feature'], rotation=90)
plt.tight_layout()
plt.show()

# Definir un umbral para la importancia promedio
importance_threshold = 0.003  # Ajusta este valor según tus necesidades

# Identificar características a mantener
features_to_keep = feature_importances_df[feature_importances_df['XGBoostImportanceNorm'] >= importance_threshold]['Feature'].tolist()

print(f"Número de características a mantener: {len(features_to_keep)}")
print("Características a eliminar:")
print(set(all_feature_names) - set(features_to_keep))

In [ ]:
# Distribution of top features
top_features = feature_importances_df['Feature'].head(10).tolist()
available_features = [f for f in top_features if f in X_train.columns]
X_train[available_features].hist(figsize=(15, 10), bins=50)
plt.tight_layout()
plt.show()

# Prepare data for RFECV
X_train_preprocessed = pipeline_xgb.named_steps['preprocessor'].transform(X_train)

# Check the shape of the preprocessed data
print(f"Shape of preprocessed data: {X_train_preprocessed.shape}")

# If X_train_preprocessed is a 2D array with only one column, we need to reshape it
if X_train_preprocessed.ndim == 2 and X_train_preprocessed.shape[1] == 1:
    X_train_preprocessed = X_train_preprocessed.reshape(-1)

# Now check if it's a 1D array
if X_train_preprocessed.ndim == 1:
    X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=['preprocessed_feature'])
    
    # Recursive Feature Elimination with Random Forest
    rfe_selector_rf = RFECV(estimator=RandomForestClassifier(random_state=random_state), step=1, cv=5, scoring='roc_auc')
    rfe_selector_rf = rfe_selector_rf.fit(X_train_preprocessed, y_train)

    # Recursive Feature Elimination with XGBoost
    rfe_selector_xgb = RFECV(estimator=XGBClassifier(random_state=random_state), step=1, cv=5, scoring='roc_auc')
    rfe_selector_xgb = rfe_selector_xgb.fit(X_train_preprocessed, y_train)

    print('Optimal number of features (Random Forest model): {}'.format(rfe_selector_rf.n_features_))
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(rfe_selector_rf.grid_scores_) + 1), rfe_selector_rf.grid_scores_)
    plt.title('Feature Number Selection (Random Forest)')
    plt.xlabel("Number of Features Selected")
    plt.ylabel("CV Score (ROC AUC)")
    plt.show()

    print('Optimal number of features (XGBoost model): {}'.format(rfe_selector_xgb.n_features_))
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(rfe_selector_xgb.grid_scores_) + 1), rfe_selector_xgb.grid_scores_)
    plt.title('Feature Number Selection (XGBoost)')
    plt.xlabel("Number of Features Selected")
    plt.ylabel("CV Score (ROC AUC)")
    plt.show()
else:
    print("Preprocessed data is not in the expected format. Please check the preprocessor output.")
    print(f"Data type: {type(X_train_preprocessed)}")
    print(f"Shape: {X_train_preprocessed.shape}")
    print(f"Sample of data: {X_train_preprocessed[:5]}")